In [1]:
import requests, json
import csv
import re

api_key='P3XzcDn6wM4Q_bV9RHL6T8BTFsRHwDy2zMOH47qys0ra5o3AhIHjqykSxZH8zBrMatiZ6bQSXkJ-Tq3XT6NZ-WiPkiyCHLWjFO_55W9M15MLFY3c4yliytf-XNxEYHYx'
headers = {'Authorization': 'Bearer %s' % api_key}

In [2]:
#Step 1. Use Business Search to get Name, ID, Rating, Categories... of Top 100 Restaurant 

#long common subsquence

In [3]:
def lcs_algo(S1, S2, m, n):
    L = [[0 for x in range(n+1)] for x in range(m+1)]

    # Building the mtrix in bottom-up way
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif S1[i-1] == S2[j-1]:
                L[i][j] = L[i-1][j-1] + 1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])

    index = L[m][n]

    lcs_algo = [""] * (index+1)
    lcs_algo[index] = ""

    i = m
    j = n
    while i > 0 and j > 0:

        if S1[i-1] == S2[j-1]:
            lcs_algo[index-1] = S1[i-1]
            i -= 1
            j -= 1
            index -= 1

        elif L[i-1][j] > L[i][j-1]:
            i -= 1
        else:
            j -= 1
            
    # Printing the sub sequences
#     print("S1 : " + S1 + "\nS2 : " + S2)
#     print("LCS: " + "".join(lcs_algo))
    return "".join(lcs_algo)

def lcs(s1, s2):
    return lcs_algo(s1, s2, len(s1), len(s2))

S1 = "ACADB"
S2 = "CBDA"
m = len(S1)
n = len(S2)
lcs(S1, S2)

'CB'

In [4]:
def request_json(rest, city):
    url='https://api.yelp.com/v3/businesses/search'

    # In the dictionary, term can take values like food, cafes or businesses
    params = {'term':rest, 'location':city}

    req=requests.get(url, params=params, headers=headers)

    # proceed only if the status code is 200
    #print('The status code is {}'.format(req.status_code))
    
    #print(req.text)
    
    return json.loads(req.text)

In [5]:
def parse_json(obj, query_name):
    rests_json = obj['businesses']
    if len(rests_json) == 0:
        return 'NaN', 'NaN', 'NaN'
    rest_names = [rest['name'] for rest in rests_json]
    lcs_results = [len(lcs(name, query_name)) for name in rest_names]
    max_index = lcs_results.index(max(lcs_results))
    rest_json = rests_json[max_index]
    return rest_json['id'], [x['alias'] for x in rest_json['categories']], rest_json['rating']
    # print(rest['id'], len(lcs(rest['name'], rest_name)), rest['name'], rest['categories'], rest['location']['city'],rest['rating'])

In [125]:
parse_json(json_obj, "Carmine's (Times Square)")

('OCTiJwvjoK81WoDwsTkFvA', ['italian', 'salad', 'seafood'], 4.0)

In [6]:
import pandas as pd

#Use pandas to save get csv file
# dfr = pd.read_csv( r"C:\Users\csucuogl\Downloads\Top 100 Independent Restautant- 2020.csv")
#dfr = pd.read_csv('/Users/cahtl/Desktop/Top_100_Independent_Restautant-2020.csv')
df = pd.read_csv('/Users/yilanye/Desktop/Yelp/Top-100.csv')
df.head()

,Rank,Restaurant,Sales ($),Average Check ($),City,State,Meals Served,Yelp ID,Rating,Categories
0,1,Carmine's (Times Square),39080335,40,"New York, NY",NY,469803,NaN,NaN,NaN
1,2,The Boathouse Orlando,35218364,43,"Orlando, FL",FL,820819,NaN,NaN,NaN
2,3,Old Ebbitt Grill,29104017,33,"Washington, DC",DC,892830,NaN,NaN,NaN
3,4,LAVO Italian Restaurant & Nightclub,26916180,90,"New York, NY",NY,198500,NaN,NaN,NaN
4,5,Bryant Park Grill & Cafe,26900000,62,"New York, NY",NY,403000,NaN,NaN,NaN


In [9]:
for i, (name, city) in enumerate(zip(df['Restaurant'], df['City'])):
#     print(name, city)
    json_obj = request_json(name, city)
    id, categories, rating = parse_json(json_obj, name)
    df.at[i, 'Yelp ID'] = id
    df.at[i, 'Rating'] = rating
    df.at[i, 'Categories'] = categories
    print(id, categories, rating)
    
df.head()

ValueError: could not convert string to float: '22nKUyCIbpnzR6R3_g1ptQ'

In [8]:
df.to_csv('/Users/yilanye/Desktop/Yelp/Top100_Results.csv')

In [119]:
url='https://api.yelp.com/v3/businesses/search'

    # In the dictionary, term can take values like food, cafes or businesses
params = {'term':'italian', 'location':'new york city'}

req=requests.get(url, params=params, headers=headers)

json_obj = json.loads(req.text)

businesses = json_obj['businesses']

rests = [rest for rest in businesses if rest['rating'] >= 4.0]

for x in rests:
    print(x['name'])

Paesano of Mulberry Street
Giano Restaurant
Lillo Cucina Italiana
Da Andrea
Supper
Rubirosa
L'Artusi
Olio e Piú
Piccola Cucina Osteria
Aurora
AlMar
Lupa
Trattoria Trecolori
Il Corallo Trattoria
Mille Nonne
Fragole Ristorante
Tony's Di Napoli
La Lanterna Di Vittorio
Barbalu Restaurant
Malatesta Trattoria


In [ ]:
def parse_json(obj, query_name):
    rests_json = obj['businesses']
    if len(rests_json) == 0:
        return 'NaN', 'NaN', 'NaN'
    rest_names = [rest['name'] for rest in rests_json]
    lcs_results = [len(lcs(name, query_name)) for name in rest_names]
    max_index = lcs_results.index(max(lcs_results))
    rest_json = rests_json[max_index]
    return rest_json['id'], [x['alias'] for x in rest_json['categories']], rest_json['rating']
    # print(rest['id'], len(lcs(rest['name'], rest_name)), rest['name'], rest['categories'], rest['location']['city'],rest['rating'])